In [45]:
import numpy as np
import gym
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import RMSprop
from keras import optimizers

from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy,LinearAnnealedPolicy, BoltzmannQPolicy, GreedyQPolicy, MaxBoltzmannQPolicy, BoltzmannGumbelQPolicy, SoftmaxPolicy
from rl.core import Processor

In [46]:
def create_agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [47]:
class TaxiProcessor(Processor):
    def process_observation(self, observation):
        one_hot = np.zeros(500)
        one_hot[observation] = 1
        return one_hot

    def process_reward(self, reward):
        return reward

In [48]:
env = gym.make("Taxi-v3")
print("No of state = ", env.observation_space.n)
print("No of actions =", env.action_space.n)
model = create_agent(500, env.action_space.n)
model.summary()

No of state =  500
No of actions = 6
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 500)               0         
                                                                 
 dense_24 (Dense)            (None, 24)                12024     
                                                                 
 dense_25 (Dense)            (None, 24)                600       
                                                                 
 dense_26 (Dense)            (None, 24)                600       
                                                                 
 dense_27 (Dense)            (None, 6)                 150       
                                                                 
Total params: 13,374
Trainable params: 13,374
Non-trainable params: 0
_________________________________________________________________


In [49]:
memory = SequentialMemory(limit=800000, window_length=1)
processor = TaxiProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)

#policy = LinearAnnealedPolicy(MaxBoltzmannQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)
dqn = DQNAgent(model=model, nb_actions=env.action_space.n, processor=processor, memory=memory, nb_steps_warmup=1000, gamma=0.99, policy=policy, enable_double_dqn=False, target_model_update=5e-4)



In [50]:
optimizer = tf.keras.optimizers.legacy.RMSprop(lr=1e-3)
optimizer1 = optimizers.Adam(lr=1e-3)
dqn.compile(optimizer1, metrics=['mae'])
dqn.fit(env, nb_steps=250000, visualize=False, verbose=1)

Training for 250000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: -2.5015
51 episodes - episode_reward: -490.137 [-911.000, -149.000] - loss: 0.240 - mae: 4.084 - mean_q: -1.838 - mean_eps: 0.478 - prob: 1.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: -1.1281
50 episodes - episode_reward: -224.480 [-272.000, -150.000] - loss: 0.103 - mae: 7.173 - mean_q: -5.349 - mean_eps: 0.050 - prob: 1.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: -1.1998
52 episodes - episode_reward: -229.058 [-1028.000, -14.000] - loss: 0.231 - mae: 10.159 - mean_q: -8.705 - mean_eps: 0.050 - prob: 1.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 85s 9ms/step - reward: -1.1326
52 episodes - episode_reward: -220.269 [-263.000, -13.000] - loss: 0.370 - mae: 12.571 - mean_q: -11.216 - mea

In [51]:
env.reset()
res = dqn.test(env, nb_episodes=100, visualize=False)


Testing for 100 episodes ...
Episode 1: reward: 8.000, steps: 13
Episode 2: reward: 7.000, steps: 14
Episode 3: reward: 9.000, steps: 12
Episode 4: reward: 5.000, steps: 16
Episode 5: reward: 5.000, steps: 16
Episode 6: reward: 4.000, steps: 17
Episode 7: reward: 12.000, steps: 9
Episode 8: reward: 13.000, steps: 8
Episode 9: reward: 8.000, steps: 13
Episode 10: reward: 12.000, steps: 9
Episode 11: reward: 10.000, steps: 11
Episode 12: reward: 7.000, steps: 14
Episode 13: reward: 11.000, steps: 10
Episode 14: reward: 7.000, steps: 14
Episode 15: reward: 6.000, steps: 15
Episode 16: reward: 11.000, steps: 10
Episode 17: reward: 5.000, steps: 16
Episode 18: reward: 8.000, steps: 13
Episode 19: reward: 9.000, steps: 12
Episode 20: reward: 8.000, steps: 13
Episode 21: reward: 9.000, steps: 12
Episode 22: reward: 4.000, steps: 17
Episode 23: reward: 10.000, steps: 11
Episode 24: reward: 4.000, steps: 17
Episode 25: reward: 12.000, steps: 9
Episode 26: reward: 6.000, steps: 15
Episode 27: re

In [52]:
res.history
print(np.average(res.history['episode_reward']))

7.9
